In [0]:
# Mount Google Drive (If using Colab)

from google.colab import drive
drive.mount("drive")

In [0]:
# Install required Packages from PyPI

!pip install face-detection

In [0]:
# Import Required Packages

import os
import numpy as np
import cv2
from keras.applications import resnet50
from keras.models import Model
from keras.layers import Dense,Dropout,AveragePooling2D,Flatten,Adam
from keras.preprocessing.image import load_img,img_to_array
from sklearn.model_selection import train_test_split

In [0]:
# Initialize a Face Detector (Confidence Threshold can be adjusted to detect more Clear Faces)

detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

In [0]:
# Load Pretrained ResNet50 Model without Last few Layers and Freeze them

base_network  = resnet50.ResNet50(input_shape = (224,224,3),weights='imagenet',include_top=False)
for layer in base_network.layers:
  layer.trainaibale = False

In [0]:
# Define the Face Mask Classifier Model by adding a few Layers on top of the ResNet50 Pretrained Model

classifier_network = base_network.output
classifier_network = AveragePooling2D(pool_size=(7, 7),name = "Average_Pool_Final")(classifier_network)
classifier_network = Flatten(name = "Flatten_Final")(classifier_network)
classifier_network = Dense(128, activation="relu",name = "Dense_Final")(classifier_network)
classifier_network = Dropout(0.5,name = "Dropout_Final")(classifier_network)
classifier_network = Dense(1, activation="sigmoid", name = "Sigmoid_Classifier" )(classifier_network)

mask_classifier = Model(inputs=base_network.input, outputs=classifier_network)

In [0]:
# Set Paths to the Datasets (Masked and Unmasked Faces)

Masked_Path = "drive/My Drive/Social_Distancing_with_AI/Training_Data/with_mask"
UnMasked_Path = "drive/My Drive/Social_Distancing_with_AI/Training_Data/without_mask"

In [0]:
# Load and Prepare the Data for Training 

faces = []
labels = []

paths = [Masked_Path,UnMasked_Path]

loaded = 0 

for i in range(2):

  for file_name in os.listdir(paths[i]):

    try:
      
      image = cv2.imread(os.path.join(paths[i],file_name))

      detections = detector.detect(image[:,:,::-1])

      for j in range(len(detections)):
    
        face = image[int(detections[j][1]):int(detections[j][3]),
                     int(detections[j][0]):int(detections[j][2]),::-1]
        face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_NEAREST)
        face = resnet50.preprocess_input(face)
        faces.append(face)
        labels.append(i)

      loaded+=1

      if loaded%100==0:
        print(str(loaded)+" images loaded successfuly !")
    
    except:
      print("Error loading "+str(os.path.join(paths[i],file_name))+" !")

faces = np.array(faces)
labels = np.array(labels)
labels = labels.reshape((labels.shape[0],1))

In [0]:
# Set the Hyper-Parameters

alpha = 0.0001
n_epochs = 5
mini_batch_size = 32

In [0]:
# Compile the Model

opt = Adam(learning_rate=alpha,decay = alpha/n_epochs)
mask_classifier.compile(optimizer=opt,loss="binary_crossentropy",metrics = ['accuracy'])

In [0]:
# Perform Train-Test Split

x_train,x_test,y_train,y_test = train_test_split(faces,labels,test_size = 0.2)

In [0]:
# Train the Model on the Loaded Data

mask_classifier.fit(x_train,y_train,epochs=n_epochs,batch_size=mini_batch_size)

In [0]:
# Evaluate on Test Set

mask_classifier.evaluate(x_test,y_test)

In [0]:
# Save the Trained Weights to Disk

mask_classifier.save("drive/My Drive/Social_Distancing_with_AI/ResNet50_Classifier.h5")